In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne 
from nice.markers import (KolmogorovComplexity, TimeLockedContrast, PowerSpectralDensityEstimator, 
                          PowerSpectralDensity, SymbolicMutualInformation, PermutationEntropy)

import pycsd

Using MNE with API > 0.19


In [1]:
def all_markers(epochs, tmin, tmax):
        """
        Computes all ther markers for given epochs.
        epochs: the epochs from which to compute the markers
        
        """
        
        

        # =============================================================================
        # SPECTRAL MARKERS
        # =============================================================================
          #PowerSpectralDensityL
        psds_params = dict(n_fft=4096, n_overlap=100, n_jobs='auto', nperseg=128)
        base_psd = PowerSpectralDensityEstimator(
            psd_method='welch', tmin=tmin, tmax=tmax, fmin=1., fmax=45.,
            psd_params=psds_params, comment='default')


            #alpha normalized
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=True, comment='alphan')
        alpha.fit(epochs)
        from scipy.stats import trim_mean
        def trim_mean80(a, axis=0):
            return trim_mean(a, proportiontocut=.1, axis=axis)


        reduction_func = [{'axis': 'frequency', 'function': np.sum},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]
        #alpha normalized
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=True, comment='alpha')
        alpha.fit(epochs)
        dataalpha_n = alpha._reduce_to(reduction_func, target='epochs', picks=None)

        #alpha
        alpha = PowerSpectralDensity(estimator=base_psd, fmin=8., fmax=13.,normalize=False, comment='alpha')
        alpha.fit(epochs)
        dataalpha = alpha._reduce_to(reduction_func, target='epochs', picks=None)

        #delta normalized
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4.,normalize=True, comment='delta')
        delta.fit(epochs)
        datadelta_n = delta._reduce_to(reduction_func, target='epochs', picks=None)


        #delta
        delta = PowerSpectralDensity(estimator=base_psd, fmin=1., fmax=4,normalize=False, comment='delta')
        delta.fit(epochs)
        datadelta = delta._reduce_to(reduction_func, target='epochs', picks=None)

        #theta normalized
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8.,normalize=True, comment='theta')
        theta.fit(epochs)
        datatheta_n = theta._reduce_to(reduction_func, target='epochs', picks=None)


        #theta
        theta = PowerSpectralDensity(estimator=base_psd, fmin=4., fmax=8,normalize=False, comment='theta')
        theta.fit(epochs)
        datatheta = theta._reduce_to(reduction_func, target='epochs', picks=None)

        #gamma normalized
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45.,normalize=True, comment='gamma')
        gamma.fit(epochs)
        datagamma_n = gamma._reduce_to(reduction_func, target='epochs', picks=None)


        #gamma
        gamma = PowerSpectralDensity(estimator=base_psd, fmin=30., fmax=45,normalize=False, comment='theta')
        gamma.fit(epochs)
        datagamma = gamma._reduce_to(reduction_func, target='epochs', picks=None)

        #beta normalized
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30.,normalize=True, comment='beta')
        beta.fit(epochs)
        databetaa_n = beta._reduce_to(reduction_func, target='epochs', picks=None)


        #beta
        beta = PowerSpectralDensity(estimator=base_psd, fmin=13., fmax=30,normalize=False, comment='beta')
        beta.fit(epochs)
        databeta = beta._reduce_to(reduction_func, target='epochs', picks=None)


        # =============================================================================
        # INFORMATION THEORY MARKERS
        # =============================================================================

        komplexity = KolmogorovComplexity(tmin=tmin, tmax=tmax, backend='openmp')
        komplexity.fit(epochs)
        komplexityobject=komplexity.data_ ###Object to save, number of channels*number of epochs, it's ndarray
        reduction_func= [{'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]


        datakomplexity = komplexity._reduce_to(reduction_func, target='epochs', picks=None)


        p_e = PermutationEntropy(tmin=tmin, tmax=tmax)
        p_e.fit(epochs)
        p_eobject = p_e.data_
        datap_e = p_e._reduce_to(reduction_func, target='epochs', picks=None)

        # =============================================================================
        # wSMI MARKERS
        # =============================================================================
        wSMI = SymbolicMutualInformation(tmin=tmin, tmax=tmax, kernel=3, tau=8, backend="openmp",
                     method_params=None, method='weighted', comment='default')
        wSMI.fit(epochs)
        wSMIobject = wSMI.data_

        reduction_func= [{'axis': 'channels_y', 'function': np.median},
             {'axis': 'channels', 'function': np.mean},
             {'axis': 'epochs', 'function': trim_mean80}]
        
        datawSMI = wSMI._reduce_to(reduction_func, target='epochs', picks=None)
        
        
        return {'wSMI':datawSMI, 'p_e':datap_e, 'k':datakomplexity, 'b':databeta,'b_n':databetaa_n, 'g':datagamma, 'g_n':datagamma_n, 't':datatheta,'t_n': datatheta_n , 'd':datadelta,
        'd_n':datadelta_n, 'a_n':dataalpha_n, 'a':dataalpha}


In [4]:
epoch_type = 'evoked'
# epoch_type = 'pseudo-rs'


all_participants = ['VP07','VP08','VP09', 'VP10','VP11','VP12','VP13','VP14','VP18','VP19','VP20','VP22','VP23','VP24','VP25','VP26','VP27','VP28','VP29','VP30','VP31','VP32','VP33','VP35','VP36','VP37']


# path = '/media/nicolas.bruno/63f8a366-34b7-4896-a7ce-b5fb4ee78535/Nico/MW_eeg_data/minmarker/' #icm-linux
path = '/Users/nicobruno/ownCloud/MW_eeg_data/minmarker/' #nico-mac

In [12]:
participant = all_participants[7]

folder = path + participant +'/'
epochs_subtracted = mne.read_epochs(folder + participant + epoch_type + 'ar-subtracted_epo.fif')

epochs_subtracted.info['description'] = 'biosemi/64'

epochs =  epochs_subtracted.pick_types(eeg = True)

SC = all_markers(epochs_subtracted['SC/go/correct'], 0, 0.6)

SC['wSMI']

Reading /Users/nicobruno/ownCloud/MW_eeg_data/minmarker/VP14/VP14evokedar-subtracted_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
343 matching events found
No baseline correction applied
0 projection items activated
Cannot autodetect number of jobs
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency'

array([0.10179828, 0.11630554, 0.1126354 , 0.10469372, 0.10905155,
       0.10644897, 0.11224332, 0.11383004, 0.11605613, 0.10797002,
       0.10574864, 0.11211519, 0.10567306, 0.10011177, 0.11088704,
       0.0964164 , 0.10406492, 0.1213941 , 0.1129302 , 0.11298483,
       0.10516367, 0.11328578, 0.10530567, 0.10990231, 0.10293791,
       0.11904957, 0.11069333, 0.10464739, 0.11021706, 0.10376165,
       0.10915305, 0.10886497, 0.11667688, 0.11133039, 0.1134475 ,
       0.11078393, 0.10729881, 0.11194112, 0.10716575, 0.11022904,
       0.10962275, 0.11063663, 0.11370081, 0.11015516, 0.10807509,
       0.10722241, 0.10996234, 0.11902649, 0.10232114, 0.10094935,
       0.10757453, 0.11600436, 0.10453761, 0.10676951, 0.10632507,
       0.10247258, 0.10893619, 0.10820026, 0.10898966, 0.10408302,
       0.10570307, 0.11152551, 0.10868518, 0.09920561, 0.10213914,
       0.10583251, 0.10506358, 0.10940228, 0.10231239, 0.11040684,
       0.11052875, 0.10947742, 0.09652368, 0.10923338, 0.10963

In [14]:
for i in all_participants:
    participant = i

    folder = path + participant +'/'
    epochs = mne.read_epochs(folder + participant + epoch_type + 'ar-subtracted_epo.fif')
    
    epochs.info['description'] = 'biosemi/64' #necessary for wSMI
    
    epochs =  epochs.pick_types(eeg = True) #EOGs break everything
    
    try:
        pd.DataFrame.from_dict(all_markers(epochs['SC/go/correct'], 0, 0.6)).to_csv(folder +  participant + '_' + epoch_type + '_' + 'SC'+'.csv')
    except:
        pass

    try:
        pd.DataFrame.from_dict(all_markers(epochs['PC/go/correct'], 0, 0.6)).to_csv(folder +  participant + '_' + epoch_type + '_' + 'PC'+'.csv')
    except:
        pass


Reading /Users/nicobruno/ownCloud/MW_eeg_data/minmarker/VP07/VP07evokedar-subtracted_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
123 matching events found
No baseline correction applied
0 projection items activated
Cannot autodetect number of jobs
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency'

/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: divide by zero encountered in log
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))
/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: invalid value encountered in multiply
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))


Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
Cannot autodetect number of jobs
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['freq

/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: divide by zero encountered in log
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))
/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: invalid value encountered in multiply
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))


Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
Cannot autodetect number of jobs
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['freq

...

Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
Cannot autodetect number of jobs
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['freq

/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: divide by zero encountered in log
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))
/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: invalid value encountered in multiply
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))


Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
Cannot autodetect number of jobs
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['freq

/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: divide by zero encountered in log
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))
/Users/nicobruno/opt/anaconda3/lib/python3.8/site-packages/nice-0.1.dev0-py3.8.egg/nice/algorithms/information_theory/permutation_entropy.py:75: RuntimeWarning: invalid value encountered in multiply
  pe = np.nan_to_num(-np.nansum(count * np.log(count), axis=1))


Reduction order for nice/marker/SymbolicMutualInformation/default: ['channels_y', 'channels']
Cannot autodetect number of jobs
Effective window size : 16.384 (s)
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/alpha: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/delta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/gamma: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/theta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['frequency', 'channels']
Reduction order for nice/marker/PowerSpectralDensity/beta: ['freq